# ULTImodel tutorial 6: Generate country attributes for international travel 

In this tutorial, the general country attributes that influence trip generation will be determined using the `AttractionFactors` methods. For trip generation, the travel volumes per country are classified into national and international (border crossing) trips using certain country attributes, which are determined in the following section using `AttractionFactors.BorderCrossingAtts`.

The inputs are:
- TAZ as `GeoDataFrame`
- International road network with edges (see tutorial3)

In [1]:
# for network creation
from ultimodel import AttractionFactors

In [2]:
# for reading and saving files etc.
import geopandas as gpd
import pandas as pd
import numpy as np
# for time tracking
from datetime import datetime

## Read input: TAZ and population

The input includes georeferenced TAZ in `EPSG:4326` with the following _required_ attributes (columns):

* __ID__ | field including a unique ID, e.g. the NUTS ID
* __Country__ | field containing the ISO-2 code of the respective country
* __population__ | field containing the population

The population attribute was added in `tutorial5`, which is why the resulting TAZ `GeoDataFrame` is imported.

Other attributes like name etc. can be added, but are not required for the following steps.

The international network edges are imported, as created in `tutorial3`.

In [3]:
# load taz from database or local
taz = gpd.GeoDataFrame.from_file('tutorial-files/tutorial5/taz-tutorial-attindex.gpkg')

# defining ID, country and geometry column names of taz
taz_id = "nuts_id"
taz_cn = "cntr_code"
taz_geo = "geometry"

taz.head()

,nuts_id,cntr_code,nuts_name,population,ind_area_count,ind_area_sum,index_nat,index_int,geometry
0,FI193,FI,Keski-Suomi,510850.572340,282,2.246628e+07,0.903992,0.889445,"MULTIPOLYGON (((26.13865 63.45759, 26.16055 63..."
1,FI194,FI,Etelä-Pohjanmaa,358466.021589,396,2.183331e+07,0.818465,0.805294,"MULTIPOLYGON (((21.64783 62.01959, 21.63880 62..."
2,FI195,FI,Pohjanmaa,340041.957401,276,1.842693e+07,0.698118,0.686884,"MULTIPOLYGON (((21.64783 62.01959, 21.52578 62..."
3,FI196,FI,Satakunta,358255.966908,390,3.533939e+07,0.919390,0.904596,"MULTIPOLYGON (((21.41993 61.04744, 21.42015 61..."
4,FI197,FI,Pirkanmaa,901022.640876,505,4.031231e+07,1.607394,1.581528,"MULTIPOLYGON (((22.83124 62.27089, 22.90118 62..."


In [4]:
countries = taz[taz_cn].unique()

We will add Polygons at the borders for Germany and Russia in this tutorial to determine the country attributes and no forget any neighbors

In [5]:
taz = pd.concat([taz, gpd.GeoDataFrame.from_file('tutorial-files/_input/DE-RU-tutorial6.gpkg')])

In [6]:
net_int = gpd.GeoDataFrame.from_file('tutorial-files/tutorial3/international-edges.gpkg')

In [7]:
path_export = 'tutorial-files/tutorial6/'

## Get country attributes

This step creates a `DataFrame` of the countries with the following attributes:

- population
- area
- share of land border
- number of neighbor countries
- number of border-crossing highways per 100km land border

In [8]:
border_att = AttractionFactors.BorderCrossingAtts(taz)

C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\geopandas\geodataframe.py:1676: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  aggregated_data = data.groupby(**groupby_kwargs).agg(aggfunc)


In [9]:
# shared border
border_att.shared_borders(inplace=True)
# border streets
border_att.border_streets(net_int, inplace=True)
# neighbor countries
border_att.count_neighbors(inplace=True)
# population and area
border_att.pop_area(inplace=True)

C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\shapely\set_operations.py:133: RuntimeWarning: invalid value encountered in intersection
  return lib.intersection(a, b, **kwargs)
C:\Users\thom_nn\.conda\envs\ultimo\lib\site-packages\shapely\set_operation

In [10]:
# remove surrounding countries (DE, RU)
border_att.country_layer = border_att.country_layer.loc[border_att.country_layer[taz_cn].isin(countries)]
# final result: country information
border_att.country_layer

,cntr_code,geometry,index,border_share,border_length,border_length_shared,border_crossings_count,neighbors,population,area
1,DK,"MULTIPOLYGON (((10.26796 55.03071, 10.27580 55...",0,0.015467,4088.436575,63.236291,5.0,2,8.396369e+06,43653.093630
2,FI,"MULTIPOLYGON (((19.96477 59.98505, 19.96479 59...",0,0.269796,8715.390137,2351.374349,136.0,3,9.547163e+06,337512.028767
3,NO,"MULTIPOLYGON (((4.88766 59.29444, 4.88027 59.2...",0,0.101108,24434.055949,2470.468692,176.0,3,8.527085e+06,322504.893423
5,SE,"MULTIPOLYGON (((12.71874 55.89266, 12.69803 55...",0,0.213012,9981.249404,2126.124196,128.0,3,1.621385e+07,449736.409751


In [47]:
# save results
border_att.country_layer.to_file(path_export + 'country-attributes' + '.gpkg', driver='GPKG')